In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140480 entries, 0 to 140479
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   time          140480 non-null  object 
 1   JS_NH3        113979 non-null  float64
 2   CS_NH3        113493 non-null  float64
 3   JS_TN         113979 non-null  float64
 4   CS_TN         113593 non-null  float64
 5   JS_LL         120761 non-null  float64
 6   CS_LL         120971 non-null  float64
 7   MCCS_NH4      120974 non-null  float64
 8   MCCS_NO3      120974 non-null  float64
 9   JS_COD        113979 non-null  float64
 10  CS_COD        120595 non-null  float64
 11  JS_SW         112368 non-null  float64
 12  CS_SW         112587 non-null  float64
 13  B_HYC_NH4     120975 non-null  float64
 14  B_HYC_XD      120975 non-null  float64
 15  B_HYC_MLSS    120975 non-null  float64
 16  B_HYC_JS_DO   120975 non-null  float64
 17  B_HYC_DO      120975 non-null  float64
 18  B_CS

# 特征处理

In [4]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [5]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW']] #




labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del all_data
gc.collect()

for f in features:
    data[f] = data[f].fillna(method='ffill')

# 提取特征的diff值

In [6]:
# tmp_features = list(features)
# add_dict = {}
# for f in tmp_features:
#     data[f'{f}_diff']  = data[f].shift()
#     features.append(f'{f}_diff')


In [7]:
# fff = ['MCCS_NO3', 'B_HYC_NH4','N_HYC_NH4']
# for f in features:
#     data[f] = data[f].fillna(method='ffill')
# for i in range(len(fff)-1):
#     for j in range(i+1, len(fff)):
#         f1 = features[i]
#         f2 = features[j]
#         # feature_jian = f"{f1}_{f2}_jian"
#         feature_radio = f"{f1}_{f2}_radio"
#         data[feature_radio] = data[f1]/data[f2]
#         # data[feature_jian] = data[f1]-data[f2]
#         features.append(feature_radio)
#         # features.append(feature_jian)

In [8]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        f_corr_name = 'rolling{}_{}_corr'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.corr().fillna(0).values
        data[f_corr_name] = train_rolling.std().fillna(0).values
        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]

        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f_corr_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')

    # for r in [5]:
    #     train_rolling = data[f].shift(15).rolling(window=r, center=False)
    #     f_mean_name = 'shift_rolling{}_{}_mean'.format(r,f)
    #     f_max_name = 'shift_rolling{}_{}_max'.format(r,f)
    #     f_min_name = 'shift_rolling{}_{}_min'.format(r,f)
    #     f_std_name = 'shift_rolling{}_{}_std'.format(r,f)
    #     data[f_mean_name] = train_rolling.mean().fillna(0).values
    #     data[f_max_name] = train_rolling.max().fillna(0).values
    #     data[f_min_name] = train_rolling.min().fillna(0).values
    #     data[f_std_name] = train_rolling.std().fillna(0).values
    #     data[f'shift_{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
    #     data[f'shift_{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
    #     data[f'shift_{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
    #
    #     if i == 0:
    #         add_featuers.append(f_mean_name)
    #         add_featuers.append(f_max_name)
    #         add_featuers.append(f_min_name)
    #         add_featuers.append(f_std_name)
    #         add_featuers.append(f'shift_{f}_{f_mean_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_max_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_min_name}_cha')
features.extend(add_featuers)

# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\hyj\AppData\Local\Temp\ipykernel_4220\2850593650.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
C:\Users\hyj\AppData\Local\Temp\ipykernel_4220\2850593650.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
C:\Users\hyj\AppData\Local\Temp\ipykernel_4220\2850593650.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

# 删除分布不均衡的特征

In [9]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
def transform(x: pd.Series, c=20):
    return np.log1p(x/c)

def inverse_transform(x: pd.Series, c = 20):
    return np.expm1(x)*c

train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
train['Label1'] = transform(train['Label1'])
label_c = 8
train['Label2'] = transform(train['Label2'], c=label_c)

In [24]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

score_list = []
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    print(np.max(tmp[label]), np.min(tmp[label]))
    test_size = 4000
    X_train = tmp[features][:-test_size]
    X_test = tmp[features][-test_size:]
    y_train = tmp[label][:-test_size]
    y_test = tmp[label][-test_size:]
    if label == "Label1":
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.1,
            n_estimators=10000,
            min_data_in_leaf=50,
            subsample = 0.8,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test), inverse_transform(test_pred))))
        print(f"score_list = {score_list}")

    else:
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.01,
            n_estimators=10000,
            min_child_weight=1,
            min_data_in_leaf=40,
            subsample = 0.4,
            feature_fraction=0.4,
            bagging_seed=11,
            reg_alpha=1,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=222
        )

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test, c=label_c), inverse_transform(test_pred, c=label_c))))
        print(f"score_list = {score_list}")

loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

35068
8.106907231653508 3.731088486252612
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.01, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.01


C:\Users\hyj\.conda\envs\python38\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hyj\.conda\envs\python38\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	training's rmse: 0.130295	training's l2: 0.0169767	valid_1's rmse: 0.108602	valid_1's l2: 0.0117944
[400]	training's rmse: 0.108607	training's l2: 0.0117954	valid_1's rmse: 0.106446	valid_1's l2: 0.0113307
[600]	training's rmse: 0.101853	training's l2: 0.010374	valid_1's rmse: 0.105489	valid_1's l2: 0.011128
[800]	training's rmse: 0.0977857	training's l2: 0.00956204	valid_1's rmse: 0.10428	valid_1's l2: 0.0108743
[1000]	training's rmse: 0.094581	training's l2: 0.00894557	valid_1's rmse: 0.103203	valid_1's l2: 0.0106508
[1200]	training's rmse: 0.0918588	training's l2: 0.00843803	valid_1's rmse: 0.102455	valid_1's l2: 0.010497
[1400]	training's rmse: 0.0896064	training's l2: 0.00802931	valid_1's rmse: 0.102002	valid_1's l2: 0.0104044
[1600]	training's rmse: 0.0876879	training's l2: 0.00768916	valid_1's rmse: 0.101633	valid_1's l2: 0.0103293
[1800]	training's rmse: 0.0859378	training's l2: 0.00738531	valid_1's rmse: 0.101295	valid_1's l2: 0.0102606
[2000]	training's rmse: 0.084352	t

C:\Users\hyj\.conda\envs\python38\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hyj\.conda\envs\python38\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	training's rmse: 0.126115	training's l2: 0.015905	valid_1's rmse: 0.112903	valid_1's l2: 0.0127471
[400]	training's rmse: 0.1057	training's l2: 0.0111725	valid_1's rmse: 0.10389	valid_1's l2: 0.0107931
[600]	training's rmse: 0.0987979	training's l2: 0.00976103	valid_1's rmse: 0.101074	valid_1's l2: 0.010216
[800]	training's rmse: 0.0948753	training's l2: 0.00900132	valid_1's rmse: 0.0990868	valid_1's l2: 0.00981819
[1000]	training's rmse: 0.0918181	training's l2: 0.00843056	valid_1's rmse: 0.0981058	valid_1's l2: 0.00962475
[1200]	training's rmse: 0.0894362	training's l2: 0.00799883	valid_1's rmse: 0.0977827	valid_1's l2: 0.00956146
[1400]	training's rmse: 0.0871566	training's l2: 0.00759627	valid_1's rmse: 0.0975145	valid_1's l2: 0.00950909
[1600]	training's rmse: 0.0851723	training's l2: 0.00725432	valid_1's rmse: 0.0973445	valid_1's l2: 0.00947595
score_list = [1306.1124924973867, 1102.426203156649]
0.8296900620619794


In [11]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8238015083098352
